In [2]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import altair as alt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
team_id = 251
team_color = "#bf5700"
season = 2023
womens = True
url = f"http://www.espn.com/womens-college-basketball/team/schedule/_/id/{team_id}/season/{season}"
stadiums_url = "https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_basketball_arenas"


# Pandas Method

In [4]:
# table = pd.read_html(url)[0]
# table.head()

# BS Method

In [5]:
page = rq.get(url)
stadium_page = rq.get(stadiums_url)

In [6]:
soup = bs(page.content)
stadium_soup = bs(stadium_page.content)
table = soup.table
stadium_table = stadium_soup.find_all(
    'table', {"class": "wikitable"})[1]
print(stadium_table)


<table class="wikitable sortable">
<tbody><tr>
<th class="unsortable">Image</th>
<th>Arena</th>
<th>City</th>
<th>State</th>
<th>Team</th>
<th>Conference</th>
<th>Capacity</th>
<th>Opened
</th></tr>
<tr>
<td><a class="image" href="/wiki/File:BC-ConteForum1.JPG"><img alt="BC-ConteForum1.JPG" data-file-height="1200" data-file-width="1600" decoding="async" height="60" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/e1/BC-ConteForum1.JPG/80px-BC-ConteForum1.JPG" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/e1/BC-ConteForum1.JPG/120px-BC-ConteForum1.JPG 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/e1/BC-ConteForum1.JPG/160px-BC-ConteForum1.JPG 2x" width="80"/></a>
</td>
<td><a href="/wiki/Conte_Forum" title="Conte Forum">Conte Forum</a>
</td>
<td><a href="/wiki/Chestnut_Hill,_Massachusetts" title="Chestnut Hill, Massachusetts">Chestnut Hill</a>
</td>
<td><a href="/wiki/Massachusetts" title="Massachusetts">MA</a>
</td>
<td><a href="/wiki/Boston_College_Eagles" tit

In [13]:
data = {'Opponent': [],
        'Attendance': [],
        'Location': [],
        # 'Capacity': [],
        'Date': [],
        # 'State':[],
        # 'City': []
        } 
for row in table.find_all('tr')[2:]:
    row_cells = row.find_all('td')
    try:
        date = row_cells[0].find('span').contents[0]
        opponent = row_cells[1].find_all('a')[1].contents[0]
        game_link = row_cells[2].find('a')['href']
        if(row_cells[1].find_all('span')[2].contents[-1] == "*"):
            location = "Neutral"
        else:
            location = "Home" if row_cells[1].find_all('span')[0].contents[0] == "vs" else "Away"
    except:
        break
    
    # find attendance number
    game_page = rq.get(game_link)
    game_soup = bs(game_page.content)
    atndnce = game_soup.find('div', {"class":"Attendance__Numbers"})
    capacity = game_soup.find('div', {"class": "Attendance__Capacity"})
    capacity = capacity.contents[4] if capacity != None else 0
    capacity = ''.join(filter(str.isdigit, capacity)
                      ) if capacity != 0 else capacity
    atndnce = atndnce.contents[4] if atndnce != None else 0
    atndnce = ''.join(filter(str.isdigit, atndnce)) if atndnce != 0 else atndnce

    if(location == "Away"):
        stadium_row = stadium_table.find('a', text=opponent)
        if(not stadium_row):
            stadium_row = stadium_table.find('a', text=opponent + "women" if womens else "men")
            if (not stadium_row):
                stadium_row = stadium_table.find('a', {"title": opponent})
        if(stadium_row):
            tds = stadium_row.parent.parent.find_all('td')
            city = tds[2].text
            state = tds[3].text
    elif (location == "Home"):
        print("todo...")
    # add current row data to dictionary
    if atndnce != 0:
        data["Opponent"].append(opponent)
        data["Attendance"].append(int(atndnce))
        data["Location"].append(location)
        data["Date"].append(date)
        # data["Capacity"].append(int(capacity))
        # data["State"].append(state) if state is not None else None
        # data['City'].append(city) if city is not None else None

df= pd.DataFrame(data)

todo...
todo...
todo...
todo...
todo...
todo...
todo...
todo...
todo...
todo...
todo...
todo...
todo...
todo...


In [14]:
df

,Opponent,Attendance,Location,Date
0,Louisiana,5658,Home,"Fri, Nov 11"
1,UConn,10167,Away,"Mon, Nov 14"
2,Marquette,376,Neutral,"Sat, Nov 19"
3,Louisville,365,Neutral,"Sun, Nov 20"
4,Rutgers,155,Neutral,"Mon, Nov 21"
5,Princeton,5137,Home,"Sun, Nov 27"
6,South Florida,5021,Home,"Fri, Dec 2"
7,Southern,4987,Home,"Sun, Dec 4"
8,Alabama State,5116,Home,"Sun, Dec 11"
9,Jackson State,1200,Neutral,"Wed, Dec 14"


Altair

In [17]:
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Opponent:O',sort = data["Opponent"], title=None),
    y=alt.Y('Attendance'),

    # column='Location:N',
    color=alt.Color('Location',
                    scale=alt.Scale(
                        domain=['Home', 'Away' ,'Neutral'],
                        range=[team_color, 'darkgray', 'lightgray'])),
    tooltip='Attendance'
).resolve_scale(
    # x='independent',
).properties(
    title="TX"
)

chart

alt.Chart(...)

In [ ]:
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Opponent:O', sort=data["Opponent"], title=None),
).properties(
    title="TX"
)

attendance_bar = chart.mark_bar().encode(
    y=alt.Y('Attendance', stack=None),

    # column='Location:N',
    color=alt.Color('Location',
                    scale=alt.Scale(
                        domain=['Home', 'Away', 'Neutral'],
                        range=[team_color, 'darkgray', 'lightgray'])))

capacity_bar = chart.mark_bar().encode(
    y=alt.Y('Capacity', stack=None),
)

alt.layer(attendance_bar, capacity_bar).resolve_scale(
    y='independent'
)

chart


alt.Chart(...)

In [ ]:
chart = alt.Chart(df).mark_bar().encode(
    alt.X('Opponent:O', sort=data["Opponent"], title=None),
    alt.Y('Attendance'),
    alt.Y2('Capacity'),

    column='Location:N',
    color=alt.Color('Location',
                    scale=alt.Scale(
                        domain=['Home', 'Away', 'Neutral'],
                        range=[team_color, 'darkgray', 'lightgray']))
).properties(
    title="TX"
).resolve_scale(x='independent')

chart


alt.Chart(...)

In [ ]:
print(chart.to_json())

Plotly

In [ ]:
df = pd.DataFrame({'x': ['Alpha', 'Beta', 'Beta', 'Beta',
                  'Gamma', 'Delta', 'Delta'], 'y': [12, 11, 9, 7, 5, 3, 1]})

fig = px.bar(df, x=df.index, y='y')
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=df.index,
        ticktext=df.x
    )
)
fig.show()